## Preambule

In [38]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from pathlib import Path
import plotly.graph_objects as go
import xarray as xr

In [39]:
path = Path.cwd()

## Input

Variable lists

In [40]:
Varlist = pd.read_excel(path / "Data/Varlist.xlsx", sheet_name = "Data")
vars = np.array(Varlist["Variable"])
cats = np.array(Varlist["Category"])
Names = np.unique(cats)
Vlists = []
for i in range(len(Names)):
    Vlists.append(vars[cats == Names[i]])

General info

In [41]:
DF = pd.read_csv(path / "Data/Models.csv")
DF_counts = pd.read_csv(path / "Data/Counts.csv", index_col=0)

Actual variances

In [42]:
XRraw = xr.open_dataset(path / "Data/XRdata.nc")
XRmeta = xr.open_dataset(path / "Data/XRmeta.nc")
XRvar = xr.open_dataset(path / "Data/Variances.nc")

## Plot

In [46]:
def rescale_size(s, max):
    s /= max
    s = s ** 0.3
    s *= 25
    return s

def get_count(var):
    if var == 'Primary Energy|Wind+Solar':
        count = str(list(DF_counts[DF_counts.Variable=="Primary Energy|Solar"].Count)[0])
    elif var == 'Secondary Energy|Electricity|Wind+Solar':
        count = str(list(DF_counts[DF_counts.Variable=="Secondary Energy|Electricity|Solar"].Count)[0])
    else:
        count = str(list(DF_counts[DF_counts.Variable==var].Count)[0])
    return count

def triangleplot(XR):
    XR = XR.sel(Time = np.arange(2030, 2101))
    vlist = XR.Variable.data
    years = XR.Time.data
    varmax = np.max(XR['Var_total']).data
    cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
            'magenta', 'red', 'darkgrey', 'blue', 'black', 'darkgreen']
    years_str = np.copy(years).astype(str)
    years_str[(years_str != '2050') & (years_str != '2100')] = ''
    fig = go.Figure()

    for i in range(10):
        a = [0+i*0.1, 0+i*0.1]
        b = [0, 1-a[0]]
        c = [1-a[0], 0]
        rgb = mpl.colors.colorConverter.to_rgb(plt.cm.get_cmap('Greys')(0.+i/15))
        col = 'rgb('+str(rgb[0])+','+str(rgb[1])+','+str(rgb[2])+')'
        fig.add_trace(go.Scatterternary(a=a, b=b, c=c, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))
        fig.add_trace(go.Scatterternary(a=b, b=a, c=c, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))
        fig.add_trace(go.Scatterternary(a=b, b=c, c=a, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))

    for v in range(len(vlist)):
        ds = XR.sel(Variable=vlist[v])
        a, b, c, s = np.array(ds[['S_c', 'S_m', 'S_z', 'Var_total']].to_array())
        s = rescale_size(s, varmax)
        fig.add_trace(go.Scatterternary(a=a, b=b, c=c,
                                        mode='markers+text+lines',
                                        name=vlist[v]+' ('+get_count(vlist[v])+')',
                                        showlegend=True,
                                        text=years,
                                        hovertemplate='%{text}<br>Var (climate): %{a}<br>Var (model): %{b} <br>Var (other): %{c}',
                                        marker={'size': s, 'color': cols[v], 'opacity': 1, 'line' :dict(width=0.5, color='black')}, line={'width': 1.5},
                                        textfont=dict(size=1, color=cols[v])))
    for v in range(len(vlist)):
        ds = XR.isel(Variable=v, Time=-1)
        a, b, c, s = np.array(ds[['S_c', 'S_m', 'S_z', 'Var_total']].to_array())
        s = rescale_size(s, varmax)
        fig.add_trace(go.Scatterternary(a=[a], b=[b], c=[c],
                                        mode='markers',
                                        name=vlist[v]+' ('+get_count(vlist[v])+')',
                                        showlegend=False,
                                        text=[years[-1]],
                                        hovertemplate='%{text}<br>Var (climate): %{a}<br>Var (model): %{b} <br>Var (other): %{c}',
                                        marker={'size': s, 'symbol': 'star', 'color': cols[v], 'opacity': 1, 'line' :dict(width=s/10, color='black')}, line={'width': 1.5},
                                        textfont=dict(size=1, color=cols[v])))
    fig.update_layout(height=900, width=1400, ternary={'sum':1,
                                                    'aaxis': {'title': 'Climate target'},
                                                    'baxis': {'title': 'Model'},
                                                    'caxis': {'title': 'Other'}},
                    font=dict(size=18))

    fig.update_layout({
    'ternary':
        {'sum':1,
        'aaxis':{'title': 'Climate target<br>', 'min': 0, 
                'linewidth':0, 'ticks':'outside',
                'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'], 'tickfont':{'size':12}},
        'baxis':{'title': 'Model &nbsp; &nbsp;', 'min': 0, 
                'linewidth':2, 'ticks':'outside',
                'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':60, 'tickfont':{'size':12}},
        'caxis':{'title': 'Other scenario<br>assumptions', 'min': 0, 
                'linewidth':2, 'ticks':'outside',
                'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':-60, 'tickfont':{'size':12}}}})
    return fig

In [47]:
figs = []
for n_i in range(len(Names)):
    figs.append(triangleplot(XRvar.sel(Variable = vars[cats == Names[n_i]])))

## Write to HTML

Normal plots

In [48]:
try:
    os.remove(path / 'Figures/VariancePlots.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

with open(path / 'Figures/VariancePlots.html', 'a') as f:
    f.write(html_w('<h1>')+'First order Variance decomposition</p></h1>')
    f.write(html_w('<body>')+'The left-bottom angle points towards the total variance minus the variance captured by the other two angles together (which is not necessarily equal to the sum). </p></body>')

    for n_i in range(len(Names)):
        if n_i > 0:
            f.write('<hr>')
        f.write(html_w('<h1>')+'Fig. '+str(n_i+1)+' - '+Names[n_i]+'</p></h1>')
        f.write(figs[n_i].to_html(full_html=False, include_plotlyjs='cdn'))